# Leonardo's Mechanical Ensemble: The First Robot Orchestra

> *"Do-Re-Mi-Fa-Sol-La-Si..."*  
> — Guido of Arezzo (Standard musical scale of the time)

## Introduction
Leonardo designed various automated musical instruments, including drums, flutes, and the famous **Viola Organista**. This notebook simulates an ensemble of these machines playing together.

We will explore:
1.  **Orchestration**: Coordinating different mechanical instruments.
2.  **Spectral Analysis**: Visualizing the frequency content of the ensemble.
3.  **Synchronization**: The challenge of keeping multiple machines in time.

---

In [ ]:
# Install the davinci-codex library
!pip install -q git+https://github.com/Shannon-Labs/davinci-codex.git

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

# Set "Brutalist Academic" plotting style
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Times New Roman', 'DejaVu Serif'],
    'axes.grid': True,
    'grid.alpha': 0.3,
    'axes.facecolor': 'white',
    'figure.facecolor': 'white',
    'text.color': 'black',
    'axes.labelcolor': 'black',
    'xtick.color': 'black',
    'ytick.color': 'black'
})


## The Physics Model

### 1. Sound Synthesis
Each instrument produces a distinct waveform:
- **Drum**: Short, broadband noise burst (Impact).
- **Flute**: Pure sine wave with breath noise (Air column).
- **Viola Organista**: Sawtooth-like wave (Bowed string).

### 2. Spectral Mixing
The total sound $S(t)$ is the sum of individual instruments $I_k(t)$:
$$ S(t) = \sum_{k} w_k \cdot I_k(t) $$
Where $w_k$ is the mix weight (volume).

In [ ]:
def generate_waveform(instrument, freq, duration, sr=44100):
    t = np.linspace(0, duration, int(sr * duration))
    
    if instrument == 'Drum':
        # Noise burst with exponential decay
        noise = np.random.normal(0, 1, len(t))
        envelope = np.exp(-10 * t)
        wave = noise * envelope
        
    elif instrument == 'Flute':
        # Sine wave + breath noise
        tone = np.sin(2 * np.pi * freq * t)
        breath = np.random.normal(0, 0.1, len(t))
        wave = tone + breath
        
    elif instrument == 'Viola Organista':
        # Sawtooth wave (rich harmonics)
        # Approximate with additive synthesis
        wave = np.zeros_like(t)
        for i in range(1, 10):
            wave += (1/i) * np.sin(2 * np.pi * freq * i * t)
            
    return t, wave

def plot_ensemble(drum_vol, flute_vol, viola_vol):
    sr = 44100
    duration = 0.1 # Short segment for visualization
    
    # Generate individual sounds
    t, drum = generate_waveform('Drum', 0, duration, sr)
    _, flute = generate_waveform('Flute', 440, duration, sr) # A4
    _, viola = generate_waveform('Viola Organista', 220, duration, sr) # A3
    
    # Mix
    mix = (drum * drum_vol) + (flute * flute_vol) + (viola * viola_vol)
    
    # FFT (Spectral Analysis)
    freqs = np.fft.rfftfreq(len(mix), 1/sr)
    mag = np.abs(np.fft.rfft(mix))
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Plot 1: Waveform
    ax1.plot(t[:1000] * 1000, mix[:1000], 'k-') # Show first 22ms
    ax1.set_title('Ensemble Waveform (Time Domain)')
    ax1.set_xlabel('Time (ms)')
    ax1.set_ylabel('Amplitude')
    ax1.set_ylim(-3, 3)
    
    # Plot 2: Spectrum
    ax2.plot(freqs, mag, 'b-')
    ax2.set_title('Frequency Spectrum (Frequency Domain)')
    ax2.set_xlabel('Frequency (Hz)')
    ax2.set_ylabel('Magnitude')
    ax2.set_xlim(0, 2000)
    
    # Annotate peaks
    if flute_vol > 0:
        ax2.axvline(440, color='g', linestyle='--', alpha=0.5, label='Flute (440Hz)')
    if viola_vol > 0:
        ax2.axvline(220, color='r', linestyle='--', alpha=0.5, label='Viola (220Hz)')
        
    ax2.legend()
    plt.show()
    
    print("Ensemble Status:")
    if drum_vol > 0: print("- Drum: Active (Rhythm)")
    if flute_vol > 0: print("- Flute: Active (Melody)")
    if viola_vol > 0: print("- Viola Organista: Active (Harmony)")

interact(plot_ensemble, 
         drum_vol=widgets.FloatSlider(min=0.0, max=1.0, step=0.1, value=0.5, description='Drum Vol'),
         flute_vol=widgets.FloatSlider(min=0.0, max=1.0, step=0.1, value=0.5, description='Flute Vol'),
         viola_vol=widgets.FloatSlider(min=0.0, max=1.0, step=0.1, value=0.5, description='Viola Vol'));

## Conclusion

Leonardo's mechanical ensemble represents an early attempt at **automated orchestration**. By combining percussive, wind, and string instruments, he sought to create a full musical experience without human performers. The challenge lay in powering all these diverse mechanisms simultaneously and keeping them in sync.